### A simple tool to extract yaml schema from your sqlite db

#### This will give more domain context to the LLM of your database

In [1]:
import sqlite3
from pathlib import Path

try:
    import yaml  # pip install pyyaml
except ImportError:
    raise SystemExit("Please install PyYAML first: pip install pyyaml")

DB_PATH = "nba.sqlite"  # <-- change to your .db filename
OUTPUT_YAML = "nba_schema.yaml"


def get_tables(conn):
    cur = conn.cursor()
    cur.execute("""
        SELECT name
        FROM sqlite_master
        WHERE type = 'table'
          AND name NOT LIKE 'sqlite_%'
        ORDER BY name
    """)
    return [row[0] for row in cur.fetchall()]


def get_columns(conn, table_name):
    cur = conn.cursor()
    cur.execute(f"PRAGMA table_info('{table_name}')")
    columns = []
    for cid, name, col_type, notnull, dflt_value, pk in cur.fetchall():
        columns.append({
            "name": name,
            "type": col_type or "TEXT",
            "nullable": not bool(notnull),
            "primary_key": bool(pk),
            "default": dflt_value,
        })
    return columns


def get_foreign_keys(conn, table_name):
    cur = conn.cursor()
    cur.execute(f"PRAGMA foreign_key_list('{table_name}')")
    fks = []
    for (
        _id,
        _seq,
        ref_table,
        from_col,
        to_col,
        on_update,
        on_delete,
        match
    ) in cur.fetchall():
        fks.append({
            "column": from_col,
            "references": {
                "table": ref_table,
                "column": to_col,
            },
            "on_update": on_update,
            "on_delete": on_delete,
            "match": match,
        })
    return fks


def build_schema(db_path: str):
    conn = sqlite3.connect(db_path)
    try:
        tables = get_tables(conn)
        schema = {"tables": {}}

        for tbl in tables:
            schema["tables"][tbl] = {
                "description": "",  # you can fill these in by hand later
                "columns": get_columns(conn, tbl),
                "foreign_keys": get_foreign_keys(conn, tbl),
            }

        return schema
    finally:
        conn.close()


def main():
    if not Path(DB_PATH).exists():
        raise SystemExit(f"Database file not found: {DB_PATH}")

    schema = build_schema(DB_PATH)

    with open(OUTPUT_YAML, "w", encoding="utf-8") as f:
        yaml.safe_dump(
            schema,
            f,
            sort_keys=False,
            allow_unicode=True,
        )

    print(f"YAML schema written to {OUTPUT_YAML}")


if __name__ == "__main__":
    main()


YAML schema written to nba_schema.yaml
